In [ ]:
import intdiff as id

sol = id.getTFScreeningFunction()

In [ ]:
print(sol)

In [ ]:
#get g(xi) for the smooth functions we just computed
g = lambda x: id.g(x,'smooth',sol)

g_approx = lambda x: id.g(x,'numeric')

print(g(1))
print(g_approx(1))

In [ ]:
#try the rutherford cross section
rsol = lambda x: [1,0]
fruth = id.ft12('smooth',rsol,xmin=1.0)

#get the f function
f = id.ft12('smooth',sol)

In [ ]:
print(f(1))
print(fruth(1))

In [ ]:
#set up a plot 
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
#got smarter about the mpl config: see mplstyles/ directory
plt.style.use('standard')

In [ ]:
#set up a plot

fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

fv = np.vectorize(f)

dx=0.1
xmax = 10
X  = np.arange(0.0001,xmax,dx)
Xh  = np.arange(1.0,xmax,dx)


ax1.plot (X, f(X), 'k-', label='f($t^{1/2}$)')
ax1.plot (X, fruth(X), 'k--', label='Rutherford')

#ax1.plot (X, tau_3v(X), 'g-', label='$\tau$$_3$')
#ax1.plot (X, ylind(X), 'k--', label='Lindhard')
#ax1.plot (X[X>0],ysingle(X[X>0]),'k-',linewidth=1)


ax1.set_yscale('linear')
ax1.set_xscale('log')
ax1.set_xlim(0.001, xmax)
ax1.set_ylim(0,0.008)
ax1.set_xlabel('dimensionless energy transfer $t^{1/2}$',**axis_font)
ax1.set_ylabel('f($t^{1/2}$)',**axis_font)
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
ax1.legend(loc=1,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
#plt.savefig('figures/figures.png')
plt.show()